In [60]:
import os
import numpy as np
import pandas as pd
import altair as alt
import sys
import json

sys.path.insert(0, '..')
from agents import PsAgent
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [61]:
lista = [i for i in os.listdir('../data/models') if i not in ['.ipynb_checkpoints']]

In [66]:
dados = []
for model in lista:
    path = f'../data/models/{model}'
    
    with open(f'{path}/args.json', 'r') as file:
        args = json.load(file)
        
    n_reflect = args['num_reflections']
    tao = args['tao']
    agent = PsAgent.load(path)
    matrix = agent.h_matrix
    for state in [0,1]:
        for colision in [0,1]:
            mult_nao_troca = 1
            mult_troca = 1
            soma_nao_troca = 0
            soma_troca = 0
            for T in range(0,1000):
                observation = [state,T,colision]
                percept = agent.percept_preprocess(observation)

                h_vector = matrix[:, percept]
                sum_ = np.sum(h_vector)
                probs = h_vector/sum_

                nao_troca = probs[0]
                troca = probs[1]
                
                _ = {
                    'model':model,
                    'reflections':args['num_reflections'],
                    'T':T/tao,
                    'state':state,
                    'colision':colision,
                    'troca':troca,
                    'nao_troca':nao_troca,
                    'mult_nao_troca':mult_nao_troca*nao_troca,
                    'mult_troca':mult_troca*troca,
                    'soma_troca':soma_troca,
                    'soma_nao_troca':soma_nao_troca
                }
                
                mult_nao_troca = troca*mult_nao_troca
                mult_troca = nao_troca*mult_troca
                
                soma_troca = soma_troca + mult_troca
                soma_nao_troca = soma_nao_troca + mult_nao_troca
                
                dados.append(_)

In [67]:
db = pd.DataFrame(dados)
db

,model,reflections,T,state,colision,troca,nao_troca,mult_nao_troca,mult_troca,soma_troca,soma_nao_troca
0,2025-03-24_05-25-14.794156,0,0.0000,0,0,0.000292,0.999708,9.997082e-01,2.918031e-04,0.000000,0.000000
1,2025-03-24_05-25-14.794156,0,0.0001,0,0,0.000303,0.999697,2.917146e-04,3.031314e-04,0.999708,0.000292
2,2025-03-24_05-25-14.794156,0,0.0002,0,0,0.000305,0.999695,8.845354e-08,3.044470e-04,1.999113,0.000292
3,2025-03-24_05-25-14.794156,0,0.0003,0,0,0.000310,0.999690,2.694530e-11,3.099509e-04,2.998214,0.000292
4,2025-03-24_05-25-14.794156,0,0.0004,0,0,0.021943,0.978057,8.178348e-15,2.191624e-02,3.997005,0.000292
...,...,...,...,...,...,...,...,...,...,...,...
523995,2025-03-24_05-42-59.599190,0,0.0995,1,1,0.500000,0.500000,1.081718e-300,1.004443e-300,1.401011,0.709886
523996,2025-03-24_05-42-59.599190,0,0.0996,1,1,0.500000,0.500000,5.408592e-301,5.022214e-301,1.401011,0.709886
523997,2025-03-24_05-42-59.599190,0,0.0997,1,1,0.500000,0.500000,2.704296e-301,2.511107e-301,1.401011,0.709886
523998,2025-03-24_05-42-59.599190,0,0.0998,1,1,0.500000,0.500000,1.352148e-301,1.255554e-301,1.401011,0.709886


In [68]:
db_mean = db.query(f'T < {100/tao}').groupby(['reflections','T','state','colision'])[
    ['troca', 'nao_troca','mult_nao_troca', 'mult_troca', 'soma_troca', 'soma_nao_troca']
].mean().reset_index()

In [69]:
alt.Chart(db_mean).mark_line().encode(
    x = 'T',
    y = alt.Y('troca').axis(format = '%'),
    row = 'state:N',
    column = 'colision',
    color = 'reflections:N'
).properties(height = 100).resolve_scale(y = 'independent')

alt.Chart(...)

In [65]:
alt.Chart(db_mean).mark_line().encode(
    x = 'T',
    y = alt.Y('nao_troca').axis(format = '%'),
    row = 'state:N',
    column = 'colision',
    color = 'reflections:N'
).properties(height = 100).resolve_scale(y = 'independent')

alt.Chart(...)

In [58]:
alt.Chart(db_mean).mark_line().encode(
    x = 'T',
    y = alt.Y('mult_troca').axis(format = '%'),
    row = 'state:N',
    column = 'colision',
    color = 'reflections:N'
).properties(height = 100).resolve_scale(y = 'independent')

alt.Chart(...)

In [59]:
alt.Chart(db_mean).mark_line().encode(
    x = 'T',
    y = alt.Y('mult_nao_troca').axis(format = '%'),
    row = 'state:N',
    column = 'colision',
    color = 'reflections:N'
).properties(height = 100).resolve_scale(y = 'independent')

alt.Chart(...)

In [50]:
alt.Chart(db_mean).mark_line().encode(
    x = 'T',
    y = alt.Y('soma_troca').axis(format = '%'),
    row = 'state:N',
    column = 'colision',
    color = 'reflections:N'
).properties(height = 100).resolve_scale(y = 'independent')

alt.Chart(...)

In [51]:
alt.Chart(db_mean).mark_line().encode(
    x = 'T',
    y = alt.Y('soma_nao_troca').axis(format = '%'),
    row = 'state:N',
    column = 'colision',
    color = 'reflections:N'
).properties(height = 100).resolve_scale(y = 'independent')

alt.Chart(...)